In [20]:
from xmitgcm import open_mdsdataset
import numpy as np
import xgcm
import xarray as xr
import random

In [77]:
seed= 4
perc_train = 80
perc_valid = 10
perc_test = 100-perc_train-perc_valid

#-----------
path_test = '/mnt/jake/bruno/ML/run_ijtest/'

Future steps
3. train NN

# Dataset

In [ ]:
data = open_mdsdataset(path_test ,prefix=['iceDiag'])
metrics = {
    ('X',): ['dxC', 'dxG'], # X distances
    ('Y',): ['dyC', 'dyG'], # Y distances
    ('X', 'Y'): ['rA', 'rAz', 'rAs', 'rAw'] # Areas
}
grid = xgcm.Grid(data, metrics = metrics)

In [15]:
data['SIdxA'] = grid.interp(grid.derivative(data.SIarea, 'X'), 'X')
data['SIdyA'] = grid.interp(grid.derivative(data.SIarea, 'Y'), 'Y')

data['SIdxh'] = grid.interp(grid.derivative(data.SIheff, 'X'), 'X')
data['SIdyh'] = grid.interp(grid.derivative(data.SIheff, 'Y'), 'Y')

data['SIdxe11'] = grid.interp(grid.derivative(data.SIeps11, 'X'), 'X')
data['SIdxe12'] = grid.interp(grid.derivative(data.SIeps12, 'X'), 'X')
data['SIdxe22'] = grid.interp(grid.derivative(data.SIeps22, 'X'), 'X')

data['SIdye11'] = grid.interp(grid.derivative(data.SIeps11, 'Y'), 'Y')
data['SIdye12'] = grid.interp(grid.derivative(data.SIeps12, 'Y'), 'Y')
data['SIdye22'] = grid.interp(grid.derivative(data.SIeps22, 'Y'), 'Y')

data['SIDiv_sig_1'] = grid.interp(grid.derivative(data.SIsig11, 'X'), 'X') + grid.interp(grid.derivative(data.SIsig12, 'Y'), 'Y')
data['SIDiv_sig_2'] = grid.interp(grid.derivative(data.SIsig12, 'X'), 'X') + grid.interp(grid.derivative(data.SIsig22, 'Y'), 'Y')

# Test-train-valid

In [52]:
def shuffle_my_array(seed, N, Array, symb, ds):
    rng = np.random.default_rng(seed)
    A = Array.values.reshape((N,))
    rng.shuffle(A)
    ds[symb] = (('n'), A)
    return ds

In [74]:
def generate_data_xarray(data, seed, perc_train, perc_valid, perc_test):
    N = data.time.shape[0]*data.YC.shape[0]*data.XC.shape[0]
    ds = xr.Dataset(coords = {'n':np.arange(N)})

    ds = shuffle_my_array(seed, N, data.SIheff, 'h', ds)
    ds = shuffle_my_array(seed, N, data.SIdxh, 'dxh', ds)
    ds = shuffle_my_array(seed, N, data.SIdyh, 'dyh', ds)
    ds = shuffle_my_array(seed, N, data.SIdxA, 'dxA', ds)
    ds = shuffle_my_array(seed, N, data.SIdyA, 'dyA', ds)
    ds = shuffle_my_array(seed, N, data.SIarea, 'A', ds)
    ds = shuffle_my_array(seed, N, data.SIeps11, 'e11', ds)
    ds = shuffle_my_array(seed, N, data.SIeps12, 'e12', ds)
    ds = shuffle_my_array(seed, N, data.SIeps22, 'e22', ds)
    ds = shuffle_my_array(seed, N, data.SIdxe11, 'dxe11', ds)
    ds = shuffle_my_array(seed, N, data.SIdxe12, 'dxe12', ds)
    ds = shuffle_my_array(seed, N, data.SIdxe22, 'dxe22', ds)
    ds = shuffle_my_array(seed, N, data.SIdye11, 'dye11', ds)
    ds = shuffle_my_array(seed, N, data.SIdye12, 'dye12', ds)
    ds = shuffle_my_array(seed, N, data.SIdye22, 'dye22', ds)
    ds = shuffle_my_array(seed, N, data.SIDiv_sig_1, 'div_sig_1', ds)
    ds = shuffle_my_array(seed, N, data.SIDiv_sig_2, 'div_sig_2', ds)
    ds_train = ds.sel(n = slice(0, int(ds.n.shape[0]*perc_train/100)-1))
    ds_valid = ds.sel(n = slice(int(ds.n.shape[0]*perc_train/100), int(ds.n.shape[0]*(perc_train+perc_valid)/100)-1))
    ds_test = ds.sel(n = slice(int(ds.n.shape[0]*(perc_train+perc_valid)/100), ds.n.shape[0]-1))
    return ds_train, ds_valid, ds_test

In [66]:
ds_train, ds_valid, ds_test = generate_data_xarray(data, seed, perc_train, perc_valid, perc_test)

# NN